# STN Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim
import numpy as np

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### DCNN Training

In [5]:
#STN targets
agfs = ['mfcc'] #'subgenres', 'mfcc'
genre = False # if not genre STN

target = None

if (genre and len(agfs) > 0) or (len(agfs) > 1):
    raise 'Error: can only train one STN target at a time'
elif genre:
    target = 'genre'
elif len(agfs) == 1:
    target = agfs[0]
else:
    raise 'Error: could not parse training target'

print('Training STN target: {}'.format(target))
    
#dataset
dataset_name = 'fma_medium'

Training STN target: mfcc


In [6]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  16
Rock                   6911
Electronic             6110
Experimental           2207
Hip-Hop                2109
Folk                   1477
Instrumental           1280
Pop                    1129
International          1004
Classical               598
Old-Time / Historic     510
Jazz                    380
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    72
Easy Listening           21
Name: genre_top, dtype: int64


In [7]:
AGF_DIM = 40
output_size = None
if target == 'genre':
    output_size = settings.num_genres
elif target in ['subgenres', 'mfcc']:
    output_size = AGF_DIM
else:
    raise 'Error: invalid target'
    
stn = STN(output_size)
stn.to(device)
stn = nn.DataParallel(stn)

In [8]:
## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(stn.parameters(), lr=0.001)
epochs = 30
batch_size = 64
valid_split = 0.2

In [9]:
trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [10]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate(stn, label_name):
    
    with torch.no_grad():
        stn.eval()
        
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1][label_name].to(device)
            
            out = stn(inputs)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        
        print('Validation f1 score: {}'.format(curr_f1))
        
        return curr_f1

In [ ]:
#Train it
for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch', epoch + 1)
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1][target].to(device)  #data[1]['{argument for agf being trained}']
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = stn(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
    if epoch % 5 == 4:
        validate(stn, target)
        stn.train()

print('Finished Training')

Starting epoch 1
[1,    50] loss: 5.362
[1,   100] loss: 4.944
[1,   150] loss: 4.677
[1,   200] loss: 4.620
[1,   250] loss: 4.543
[1,   300] loss: 4.371
[1,   350] loss: 4.356
[1,   400] loss: 4.270
[1,   450] loss: 4.215
[1,   500] loss: 4.219
[1,   550] loss: 4.174
[1,   600] loss: 4.152
[1,   650] loss: 4.134
[1,   700] loss: 4.081
[1,   750] loss: 4.057
[1,   800] loss: 4.133
[1,   850] loss: 4.115
[1,   900] loss: 4.011
[1,   950] loss: 4.014
[1,  1000] loss: 3.904
[1,  1050] loss: 3.993
[1,  1100] loss: 4.006
[1,  1150] loss: 4.004
[1,  1200] loss: 3.935
[1,  1250] loss: 3.951
[1,  1300] loss: 3.916
[1,  1350] loss: 3.810
[1,  1400] loss: 3.829
[1,  1450] loss: 3.857
[1,  1500] loss: 3.961
[1,  1550] loss: 3.826
[1,  1600] loss: 3.892
[1,  1650] loss: 3.807
[1,  1700] loss: 3.906
[1,  1750] loss: 3.693
[1,  1800] loss: 3.781
[1,  1850] loss: 3.762
[1,  1900] loss: 3.759
[1,  1950] loss: 3.732
[1,  2000] loss: 3.751
[1,  2050] loss: 3.721
[1,  2100] loss: 3.737
Starting epoch 2


### Save Model

In [ ]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
torch.save(stn, model_file)

## Load & Eval Model

In [11]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
model = torch.load(model_file)
model

DataParallel(
  (module): STN(
    (layer1): Sequential(
      (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ELU(alpha=1.0)
      (2): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    )
    (layer2): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (4): Dropout(p=0.1, inplace=False)
    )
    (layer3): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ELU(alpha=1.0)
      (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    )
    (layer4): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, mome

In [12]:
final_f1 = validate(model, target)
print(final_f1)
np.array(final_f1).tofile(f'logs/final_STN_{dataset_name}_{target}')

Validation f1 score: 0.8393451311798829
0.8393451311798829


## Previous STN Val Accuracies
FMA_SMALL
* genre: 89.X% (can't reload model to validate b/c didn't set a random seed during training and the random split of training / validation data is different so we're mixing training data into the validation set and getting  unrealistically high validation accuracies)
* mfcc: (prev. incorrect val: 97.36%)

FMA_MED
* genre: 89.3%
* mfcc: 83.9%
